# 1. Data Processing

In [42]:
# import the libraries 
import numpy as np
import pandas as pd 
from ortools.linear_solver import pywraplp

In [43]:
def input(filename):
    with open(filename) as f:
        N, D, a, b = [int(x) for x in f.readline().split()]
        F = [[0 for _ in range(D)] for _ in range(N)]
        # F is a matrix of size NxD, where F[i][j] = 1 is employ i has a day off on j-th day, else 0
        for i in range(N):
            d = [int(x) for x in f.readline().split()[:-1]]
            if d:
                F[i][d[0]-1] = 1

    return N, D, a, b, F


filename = 'data.txt'
N, D, a, b, F = input(filename)

In [44]:
# Check status to apply color for each type of status
def status_color(value):
  if value == "Rest": 
    color = 'Green'
  elif value == "Nigh":
    color = 'Red'
  else:
    color = 'White'
  return 'background-color: %s' % color

# 2. Optimization

In [45]:
solver = pywraplp.Solver('ROSTERING_MIP', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
INF = solver.infinity()

# DECISION VARIABLES
x = {}
for i in range(N):
    for j in range(D):
        for k in range(1, 5):
            x[i, j, k] = solver.IntVar(0, 1, f'x[{i}, {j}, {k}]')

z = solver.IntVar(0, D, 'z')  # z is an auxiliary variable to facilitate the objective function

In [46]:
# CONSTRAINTS
# Each employee works no more than one shift every day
for i in range(N):
    for j in range(D):
        if F[i][j] == 0:
            cstr = solver.Constraint(-INF, 1)
            for k in range(1, 5):
                cstr.SetCoefficient(x[i, j, k], 1)
            if j != 0:
                cstr.SetCoefficient(x[i, j-1, 4], 1)

In [47]:
# Employees can have a day off after having a night shift on the previous day
for i in range(N):
    for j in range(D):
        if F[i][j] == 0:
            cstr = solver.Constraint(1, 1)
            for k in range(1, 5):
                cstr.SetCoefficient(x[i, j, k], 1)
                if j != 0:
                    cstr.SetCoefficient(x[i, j-1, 4], 1)

In [48]:
# Employees will not work on their off days
for i in range(N):
    for j in range(D):
        if F[i][j] == 1:
            cstr = solver.Constraint(0, 0)
            for k in range(1, 5):
                cstr.SetCoefficient(x[i, j, k], 1)

In [49]:
# Each shift have at least alpha and beta employees at most
for j in range(D):
    for k in range(1, 5):
        cstr = solver.Constraint(a, b)
        for i in range(N):
            cstr.SetCoefficient(x[i, j, k], 1)

In [50]:
# OBJECTIVE FUNCTION
for i in range(N):  # the maximum night shift of any employee is minimize
    obj = solver.Constraint(0, INF)
    for j in range(D):
        obj.SetCoefficient(x[i, j, 4], -1)
        obj.SetCoefficient(z, 1)

In [51]:
solver.Minimize(z)
status = solver.Solve()

if __name__ == '__main__':
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal value:', solver.Objective().Value())
        for i in range(N):
            for j in range(D):
                for k in range(1, 5):
                    if x[i, j, k].solution_value() > 0:
                        print(f'Employee {i+1} works on day {j+1}, at shift {k}')
    else:
        print("No optimal solution!")

Optimal value: 1.0
Employee 1 works on day 1, at shift 2
Employee 1 works on day 2, at shift 2
Employee 1 works on day 3, at shift 2
Employee 1 works on day 4, at shift 2
Employee 1 works on day 5, at shift 1
Employee 2 works on day 1, at shift 4
Employee 2 works on day 3, at shift 3
Employee 2 works on day 5, at shift 3
Employee 3 works on day 1, at shift 3
Employee 3 works on day 2, at shift 3
Employee 3 works on day 3, at shift 1
Employee 3 works on day 4, at shift 3
Employee 3 works on day 5, at shift 2
Employee 4 works on day 1, at shift 1
Employee 4 works on day 2, at shift 3
Employee 4 works on day 3, at shift 2
Employee 4 works on day 4, at shift 3
Employee 4 works on day 5, at shift 4
Employee 5 works on day 1, at shift 2
Employee 5 works on day 2, at shift 1
Employee 5 works on day 3, at shift 4
Employee 5 works on day 5, at shift 1
Employee 6 works on day 1, at shift 2
Employee 6 works on day 2, at shift 1
Employee 6 works on day 3, at shift 3
Employee 6 works on day 4, at s

# 3. Visualization

In [52]:
# Create S to store day rest of each staff
# Definite shift > 0, so create S(N,D,5) with first col useless
S = np.full((N, D, 5), 0)

for staff in range(N):
    for day in range(D):
        for shift in range(1,5):
            # add value to matrix S full 0 
            S[staff, day, shift] = int(x[staff, day, shift].solution_value())

shifts = np.array(["Morning", "Noon", "Afternoon", "Night"])
days = np.array([f"Day {day}" for day in range(1,D+1)])
day_shifts = np.sum(S, axis=0)
day_shifts_solution = pd.DataFrame(data=day_shifts[:, 1:].T, index=shifts, columns=days)

# Visualize number staffs for each shift of day
day_shifts_solution.style.background_gradient(cmap='Pastel1')

,Day 1,Day 2,Day 3,Day 4,Day 5
Morning,3,2,2,1,2
Noon,3,2,3,2,2
Afternoon,2,3,2,3,3
Night,1,1,1,1,1


In [53]:
col = np.array([f"Staff {staff}" for staff in range(1,N+1)])
row = days
details_ip = np.full((D,N),"Rest")

for r in range(D):
  for c in range(N):
    for shift in range(1,5):
      if S[c,r,shift] == 1:
        details_ip[r,c] = shifts[shift-1]
        break

# Visualize details shift for each staff
pf_details_ip = pd.DataFrame(data = details_ip, index = row, columns = col)
pf_details_ip.style.applymap(status_color)

,Staff 1,Staff 2,Staff 3,Staff 4,Staff 5,Staff 6,Staff 7,Staff 8,Staff 9
Day 1,Noon,Nigh,Afte,Morn,Noon,Noon,Morn,Afte,Morn
Day 2,Noon,Rest,Afte,Afte,Morn,Morn,Nigh,Afte,Noon
Day 3,Noon,Afte,Morn,Noon,Nigh,Afte,Rest,Morn,Noon
Day 4,Noon,Rest,Afte,Afte,Rest,Afte,Noon,Nigh,Morn
Day 5,Morn,Afte,Noon,Nigh,Morn,Afte,Noon,Rest,Afte
